In [31]:
import pandas as pd
import os

In [ ]:
# Xử lý dữ liệu 

import pandas as pd

# Đọc dữ liệu từ file Excel vào một dictionary các DataFrame
file_A = pd.ExcelFile('data.xlsx')
data = {sheet_name: file_A.parse(sheet_name) for sheet_name in file_A.sheet_names}

# Định dạng lại cột 'Full Name' thành 'First Name' và 'Last Name'
for sheet_name, df in data.items():
    if 'Full Name' in df.columns:
        data[sheet_name][['First Name', 'Last Name']] = df['Full Name'].str.rsplit(n=1, expand=True)

# Định dạng lại cột 'Date of Birth' thành chuẩn YYYY-MM-DD
for sheet_name, df in data.items():
    if 'Date of Birth' in df.columns:
        data[sheet_name]['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce').dt.strftime('%Y-%m-%d')

# Thêm số 0 vào các số điện thoại không có số 0 đằng trước, trừ các trường trống
for sheet_name, df in data.items():
    if 'Tel.' in df.columns:
        data[sheet_name]['Tel.'] = df['Tel.'].apply(lambda x: '0' + str(x) if str(x)[0] != '0' and str(x) != '' else str(x))

# Ghi dữ liệu đã được biến đổi vào các sheet mới trong file Excel mới
with pd.ExcelWriter('data_cleaned.xlsx') as writer:
    for sheet_name, df in data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Đã thực hiện các biến đổi và ghi vào file Excel mới.")


/tmp/ipykernel_18638/2886952907.py:17: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data[sheet_name]['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce').dt.strftime('%Y-%m-%d')
/tmp/ipykernel_18638/2886952907.py:17: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data[sheet_name]['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce').dt.strftime('%Y-%m-%d')
/tmp/ipykernel_18638/2886952907.py:17: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data[sheet_name]['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce').dt.strftime('%Y-%m-%d')


Đã thực hiện các biến đổi và ghi vào file Excel mới.


In [ ]:
# Mapping dữ liệu từ file dữ liệu 


import pandas as pd
import os

# Đường dẫn đến file Excel dữ liệu A
file_A = pd.ExcelFile('data_cleaned.xlsx')

# Đường dẫn đến file Excel B (tạo file mới hoặc sử dụng simple data cdp)
file_B = 'import_data_into_cdp.xlsx'


# os.makedirs(save_directory, exist_ok=True)

sheet_names = file_A.sheet_names

# Duyệt qua từng sheet trong file A
for sheet_name in sheet_names:
    # Đọc dữ liệu từ sheet hiện tại trong file A
    df_A = pd.read_excel(file_A, sheet_name = sheet_name, dtype={'Tel.': str})

    # Tạo DataFrame mới với cấu trúc của file B
    df_B = pd.DataFrame(columns=['CRM ID', 'First Name', 'Last Name', 'Primary Email', 'Primary Phone', 'Gender', 'Age', 'Date Of Birth (YYYY-MM-DD)', 'Permanent Location', 'Living Location', 'Learning History', 'Working Companies', 'Job Titles', 'Business Industries', 'Business Contacts', 'Personal Contacts', 'Nationality', 'Current Zip Code', 'Location Code', 'Living Country', 'Living City', 'Living State', 'Media Channels', 'Content Keywords', 'Product Keywords', 'Sale Agency / Lead Source','Sale Person / Sales Representative'])

    # Mapping dữ liệu từ A sang B
    mapping = {
        'CRM ID': None,
        'First Name': 'First Name',
        'Last Name': 'Last Name',
        'Primary Email': 'Mail',
        'Primary Phone': 'Tel.',
        'Gender': None,
        'Age': None,
        'Date Of Birth (YYYY-MM-DD)': 'Date of Birth',
        'Permanent Location': None,
        'Living Location': None,
        'Learning History': 'Courses',
        'Working Companies': 'Company',
        'Job Titles': 'Job',
        'Business Industries': None,
        'Business Contacts': None,
        'Personal Contacts': None,
        'Nationality': None,
        'Current Zip Code': None,
        'Location Code': None,
        'Living Country': None,
        'Living City': None,
        'Living State': None, 
        'Media Channels': None, 
        'Content Keywords': None,  
        'Product Keywords': None,  
        'Sale Agency / Lead Source': None,  
        'Sale Person / Sales Representative': 'Source',
    }

    # Thực hiện mapping và tạo mới các cột
    for new_column, old_column in mapping.items():
        if old_column is not None and old_column in df_A.columns:
            df_B[new_column] = df_A[old_column]
        else:
            df_B[new_column] = None  # Nếu cột không tồn tại trong file A, thì tạo cột mới với giá trị None

    # Ghi dữ liệu từ DataFrame B vào sheet mới trong file B
    with pd.ExcelWriter(file_B, mode='a', if_sheet_exists='replace',) as writer:
        df_B.to_excel(writer, sheet_name= sheet_name, index=False)

print("Đã di chuyển dữ liệu từ file A sang file B và thực hiện mapping theo tên cột.")


In [ ]:
# Tạo các file CSV với tên là sheet name


# Đường dẫn đến file Excel đã xử lý ở trên (File B)
excel_file = pd.ExcelFile('import_data_into_cdp.xlsx')

# Đường dẫn lưu các file csv
save_directory = "csv_data_label"

# Lấy tên các sheet
sheet_names = excel_file.sheet_names

print(list(sheet_names))


# Create the save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)  # Handles existing and non-existent directories

# Duyệt qua từng sheet
for sheet_name in sheet_names:
    # Đọc dữ liệu của sheet hiện tại
    df = pd.read_excel(excel_file, sheet_name = sheet_name, dtype={'Primary Phone': str})
    
    # Tạo tên file dựa trên tên sheet
    file_name = sheet_name + '.csv'
    
    save_path = os.path.join(save_directory, file_name)
    # Lưu dữ liệu vào file riêng
    df.to_csv(save_path, index=False)
    
print ("Đã tạo file csv riêng cho từng sheet")